# PART D 

In [ ]:
# wiki.csv had the original cleaned and labelled links
dataframe = pd.read_csv("wiki.csv")
dataframe.head()

,Unnamed: 0,Article_Title,Label,All_Labels,Link
0,0,Absolute Value,1,1,https://en.wikipedia.org/wiki/Absolute_value
1,1,addition theorem,1,1,https://en.wikipedia.org/wiki/Addition_theorem
2,2,Adjoint of a matrix,2,2,https://en.wikipedia.org/wiki/Adjugate_matrix
3,3,Algebra of sets,2,2,https://en.wikipedia.org/wiki/Algebra_of_sets
4,4,Algorithm,2,2,https://en.wikipedia.org/wiki/Algorithm


In [ ]:
pattern_of_page = re.compile(r"([\w\d\s])+")
dataframe.reset_index()

In [ ]:
dataframe.columns

Index(['Unnamed: 0', 'Article_Title', 'Label', 'All_Labels', 'Link'], dtype='object')

In [ ]:
dataframe = dataframe[['Article_Title', 'Label']]

In [ ]:
feature_vectors= []
for index, row in dataframe.iterrows():
    try:
        feature_vectors.append(feature_vec(row["Article_Title"])[1][0])
    except:
        feature_vectors.append(nan)

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that c

In [ ]:
dataframe["feature_vector"] = feature_vectors
dataframe.dropna(inplace=True)
dataframe.reset_index()

,index,Article_Title,Label,feature_vector
0,0,Absolute Value,1,"[0.5313834027466758, 0.03690162519074138, 0.03..."
1,1,addition theorem,1,"[0.30819929816250846, 0.3595658478562599, 0.25..."
2,2,Adjoint of a matrix,2,"[0.8140220688381936, 0.030373957792469913, 0.0..."
3,3,Algebra of sets,2,"[0.06934678620247908, 0.23115595400826358, 0.0..."
4,4,Algorithm,2,"[0.5461609339173836, 0.4722444165451061, 0.069..."
...,...,...,...,...
408,457,Vector Space,1,"[0.13220983478976406, 0.06610491739488203, 0.0..."
409,458,Vieta Formula,1,"[0.05722856952873868, 0.11445713905747736, 0.0..."
410,459,Wald-Wolfowitz Run Test,1,"[0.08574929257125441, 0.08574929257125441, 0.0..."
411,461,Weighted Arithmetic Mean,1,"[0.05130630185507439, 0.07054616505072729, 0.0..."


In [ ]:
index_to_remove = []
for index, rows in dataframe.iterrows():
    if (type(rows["feature_vector"]) == nan) or (len(rows["feature_vector"]) < 100):
        index_to_remove.append(index)
dataframe = dataframe.drop(index_to_remove)
feature_vectors_list = []
for index, row in dataframe.iterrows():
    feature_vectors_list.append(np.array(row["feature_vector"]))

### Starting the construction of our neural network

In [ ]:
x_train = np.array(feature_vectors_list)
y_list = dataframe["Label"].reset_index()
y_list = np.array(dataframe["Label"])
y_list[y_list == ' '] = 0   # removing spaces

In [ ]:
y_train = []
for j in range(len(y_list)):
    array1 = np.zeros(4)
    array1[int(y_list[j])] = 1
    y_train.append(array1)
y_train = np.array(y_train)    

In [ ]:
# importing the necessary libraries
from sklearn.utils import shuffle  
x_train, y_train = shuffle(x_train, y_train)  # shuffling the data

In [ ]:
x_train_1 = x_train[:320, :]
x_valid_1 = x_train[320:, :]
x_train_1.shape, x_valid_1.shape

((320, 50), (93, 50))

In [ ]:
y_train_1 = y_train[:320, :] 
y_valid_1 = y_train[320:, :]
y_train_1.shape, y_valid_1.shape

((320, 4), (93, 4))

In [ ]:
# installing tensorflow for building our network
!pip install tensorflow
import tensorflow.keras as keras
num_categories = 4

     |████████████████████████████████| 462 kB 11.8 MB/s 


In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
from tensorflow.keras.layers import Dense
model.add(Dense(units=40, activation='relu', input_shape=(50,)))
model.add(Dense(units = 15, activation='relu'))
model.add(Dense(units = 4, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 40)                2040      
                                                                 
 dense_6 (Dense)             (None, 15)                615       
                                                                 
 dense_7 (Dense)             (None, 4)                 64        
                                                                 
Total params: 2,719
Trainable params: 2,719
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train_1, y_train_1, epochs=30, verbose=1, validation_data=(x_valid_1, y_valid_1))

Epoch 1/30
10/10 [==============================] - 0s 18ms/step - loss: 1.2425 - accuracy: 0.4781 - val_loss: 1.2692 - val_accuracy: 0.3763
Epoch 2/30
10/10 [==============================] - 0s 9ms/step - loss: 1.2389 - accuracy: 0.4531 - val_loss: 1.2677 - val_accuracy: 0.3871
Epoch 3/30
10/10 [==============================] - 0s 6ms/step - loss: 1.2331 - accuracy: 0.4688 - val_loss: 1.2664 - val_accuracy: 0.3871
Epoch 4/30
10/10 [==============================] - 0s 11ms/step - loss: 1.2285 - accuracy: 0.4969 - val_loss: 1.2650 - val_accuracy: 0.4086
Epoch 5/30
10/10 [==============================] - 0s 9ms/step - loss: 1.2266 - accuracy: 0.4719 - val_loss: 1.2654 - val_accuracy: 0.4301
Epoch 6/30
10/10 [==============================] - 0s 14ms/step - loss: 1.2212 - accuracy: 0.4750 - val_loss: 1.2657 - val_accuracy: 0.4301
Epoch 7/30
10/10 [==============================] - 0s 10ms/step - loss: 1.2165 - accuracy: 0.4875 - val_loss: 1.2666 - val_accuracy: 0.4086
Epoch 8/30
10/10

In [ ]:
feature_vectors = feature_vector("Formal proof")[1]
np.argmax(model.predict(feature_vectors))

1

In [ ]:
model.save('gnn_model') # saving our gnn model

# PART E 

In [ ]:
dic = {0:"Irrelevant", 1:"Easy", 2:"Intermediate", 3:"Advance"}
node_class = {"node":[], "labels":[]}
labelled_nodes = {}
import tensorflow
from google.colab import drive
drive.mount('/content/drive')
new_mod = tensorflow.keras.models.load_model('drive/gnn_model')

In [ ]:
for j in G.nodes:
    try:
        feature_vectors = G.nodes[j]["embedding"][1]
        print(feature_vectors)
        labe1 = np.argmax(new_mod.predict(feature_vectors)[0])
        node_class["node"].append(j)
        node_class["labels"].append(labe1)
        labelled_nodes[j] = labe1
    except:
        continue

Streaming output truncated to the last 5000 lines.
  0.02440154 0.02440154 0.04066923 0.03253538 0.16267691 0.03253538
  0.06507076 0.03253538 0.03253538 0.03253538 0.04880307 0.02440154
  0.05693692 0.03253538 0.06507076 0.03253538 0.02440154 0.08133846
  0.03253538 0.03253538 0.02440154 0.02440154 0.04066923 0.1789446
  0.03253538 0.06507076 0.05693692 0.08133846 0.02440154 0.04880307
  0.07320461 0.04066923 0.03253538 0.03253538 0.04066923 0.05693692
  0.02440154 0.04880307 0.03253538 0.04066923 0.04066923 0.39855843
  0.03253538 0.05693692 0.25214921 0.04066923 0.02440154 0.02440154
  0.04066923 0.04066923 0.02440154 0.05693692 0.03253538 0.04880307
  0.10573999 0.02440154 0.02440154 0.02440154 0.02440154 0.02440154
  0.03253538 0.10573999 0.03253538 0.02440154 0.02440154 0.03253538
  0.03253538 0.03253538 0.08133846 0.15454307 0.02440154 0.04066923
  0.05693692 0.21961383 0.07320461 0.03253538]]
[[0.03539962 0.09439898 0.03539962 0.02359974 0.03539962 0.02359974
  0.02359974 0.530

In [ ]:
node_class

{'labels': [], 'node': []}

In [ ]:
df = pd.DataFrame.from_dict(node_class)

# Developing a model which suggests topics

In [ ]:
node = input("Enter the node or the topic which you want to study : ")
topic_extracted = []
if node in G.nodes:
    for n in G.neighbors(node):
            topic_extracted.append((n, labelled_nodes[n]))   
    temp = []
    for topic1 in topic_extracted:
        for a in G.neighbors(topic1[0]):
            temp.append((nb, labelled_nodes[a]))
    topic_extracted.extend(temp)
    topic_extracted.sort(key = lambda x: x[1])
    print("Order in which you should study topics : 1 (easy), 2 (intermediate) & 3 (hard)")
    for topic, lable in topic_extracted:
        print(topic, lable)
else:
    print("No such node/topic exists")